# Train a Model to Trade on the Stock Market

![](https://ei.marketwatch.com/Multimedia/2018/01/16/Photos/ZH/MW-GB819_stocks_20180116134819_ZH.jpg?uuid=d1d56e68-faed-11e7-9db3-9c8e992d421e)

## Efficient Market Hypothesis

Future prices cannot be predicted by analyzing prices from the past. Excess returns cannot be earned in the long run by using investment strategies based on historical share prices or other historical data. Technical analysis techniques will not be able to consistently produce excess returns.

 

## Technical Analysis


In finance, technical analysis is an analysis methodology for forecasting the direction of prices through the study of past market data, primarily price and volume. The efficacy of technical  analysis is disputed by the efficient-market hypothesis which states that stock market prices are essentially unpredictable.

 

## Can a neural net learn Technical Analysis techniques to predict the market and make good trades?

Are we about to get rich?

In [1]:
#from alpha_vantage.techindicators import TechIndicators
#from alpha_vantage.timeseries import TimeSeries
import random
from indicators import *
from transform import *
import pandas_datareader as web
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import h2o
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import cufflinks as cf
from plots import *
import numpy as np
import operator
import math

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

# Set some defaults for how pandas displays
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Enter These Variables

In [2]:
stockSymbol = "GOOG"
market = "NASDAQ"
invest = 10000
window = 20

start_date = pd.datetime(2013,1,1)
split_validation = pd.datetime(2017,1,1)
split_test = pd.datetime(2018,1,1)

### Let's use a machine learning library called H2O

In [3]:
#h2o.cluster().shutdown()

In [4]:
h2o.init(max_mem_size="12G")
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,21 hours 17 mins
H2O cluster timezone:,America/Toronto
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.8
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_brett_08igme
H2O cluster total nodes:,1
H2O cluster free memory:,10.37 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


### Connect to Morningstar and Download the Historical Data  

In [5]:
df = web.DataReader(stockSymbol, 'morningstar', start=start_date)
df = df.reset_index(drop=False)
df = df.drop('Symbol', axis=1)
df.set_index('Date', inplace=True)
df = df[['Open','High', 'Low', 'Close','Volume']]


### Plot.ly is a Sweet Charting Library

In [6]:
myPlotlyKey = 'conh5EnFad0Z9Lz6mVWr'
py.sign_in(username='bretto777',api_key=myPlotlyKey)

fig = simpleStockChart(df,stockSymbol,split_validation,split_test)
py.iplot(fig, validate=False)

The draw time for this plot will be slow for clients without much RAM.


### Technical Analysis
Add technical analysis indicators used by the pros! Hopefully our model will learn how these indicators can predict tomorrow's price movements.

In [7]:
%%capture
df.reset_index(drop=False, inplace=True, col_level=0)

typical_price(df, high_col = 'High', low_col = 'Low', close_col = 'Close')
money_flow_index(df, vol_col='Volume')
rsi(df, close_col='Close')
ema(df, period=26, column='Close')
ema(df, period=12, column='Close')
macd(df, period_long=26, period_short=12, period_signal=9, column='Close')
on_balance_volume(df, trend_periods=21, close_col='Close', vol_col='Volume')
price_volume_trend(df, trend_periods=21, close_col='Close', vol_col='Volume')
bollinger_bands(df, trend_periods=20, close_col='Close')
williams_r(df, periods=14, high_col='High', low_col='Low', close_col='Close')
ultimate_oscillator(df, period_1=7,period_2=14, period_3=28, high_col='High', low_col='Low', close_col='Close')
df = df.drop(['uo_bp', 'uo_tr', 'uo_avg_1', 'uo_avg_2', 'uo_avg_3'], axis=1)

In [8]:
fig = technicalAnalysisChart(df)
py.iplot(fig, validate=False)

In [9]:
%%capture
windowTransform(df, window)

In [10]:
df['Close Tomorrow Trend'].replace(0,-1,inplace=True)    

trainingDF = df.loc[df['Date'] <= split_validation]
validationDF = df[df['Date'].isin(pd.date_range(split_validation,split_test))]
testDF = df.loc[df['Date'] > split_test]

trainingDF = trainingDF.dropna(axis=0, how='any')
validationDF = validationDF.dropna(axis=0, how='any')
testDF = testDF.fillna(value=0)

trainingDF.set_index('Date', inplace=True)
validationDF.set_index('Date', inplace=True)
testDF.set_index('Date', inplace=True)

In [11]:
train = h2o.H2OFrame(trainingDF, destination_frame="train.hex")
valid = h2o.H2OFrame(validationDF, destination_frame="valid.hex")
test = h2o.H2OFrame(testDF, destination_frame="test.hex")

train['Close Tomorrow Trend'] = train['Close Tomorrow Trend'].asfactor()
train = train.drop('Close Tomorrow', axis=1)
train = train.drop('Close Tomorrow Change', axis=1)
valid['Close Tomorrow Trend'] = valid['Close Tomorrow Trend'].asfactor()
valid = valid.drop('Close Tomorrow', axis=1)
valid = valid.drop('Close Tomorrow Change', axis=1)
test['Close Tomorrow Trend'] = test['Close Tomorrow Trend'].asfactor()
test = test.drop('Close Tomorrow', axis=1)
test = test.drop('Close Tomorrow Change', axis=1)

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = train.columns
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator as H2OPCA

pca_decomp = H2OPCA(k=50, transform="NONE", pca_method="gram_s_v_d", use_all_factor_levels=True)
pca_decomp.train(x=x, training_frame=train)
trainReduced = pca_decomp.predict(train)
trainReduced = trainReduced.cbind(train['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=valid)
validReduced = pca_decomp.predict(valid)
validReduced = validReduced.cbind(valid['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=test)
testReduced = pca_decomp.predict(test)
testReduced = testReduced.cbind(test['Close Tomorrow Trend'])

pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%


### Train the Deep Neural Net
Grid search for a good model

In [14]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

hidden_opt = [#[800],
              #[1000],
              #[2000],
              [200,200],
              #[25,25,25],
              #[50,75,75,50],
              #[75,100,100,75],
              #[50,300,300,50],
              #[1000,1000],
              #[50,50,50]
              ]
l1_opt = [0.001]
l2_opt = [0.001]
rate_opt= [0.01]
activation_opt=["Rectifier","RectifierWithDropout","Tanh", "TanhWithDropout", "Maxout", "MaxoutWithDropout"]
epochs_opt=[50,50]

hyper_parameters = {"hidden":hidden_opt, 
                    "l1":l1_opt, 
                    "l2":l2_opt,
                    "rate":rate_opt, 
                    "activation":activation_opt,
                    "epochs":epochs_opt}

search_criteria = {"strategy":"RandomDiscrete", "max_models":10, "seed":123456}



model_grid = H2OGridSearch(H2ODeepLearningEstimator(),
                           hyper_params=hyper_parameters,
                           search_criteria=search_criteria,
                           grid_id='test8')
model_grid.train(x=x, 
                 y=y,                                   
                 fast_mode=True,                   
                 stopping_rounds=10, 
                 stopping_tolerance=0,
                 stopping_metric='AUC',
                 adaptive_rate=True,                
                 training_frame=trainReduced, 
                 validation_frame=validReduced,
                 score_interval=10)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [15]:
model_gridperf1 = model_grid.get_grid(sort_by='recall', decreasing=False)
bestModel = model_gridperf1[0]          
#h2o.save_model(model=bestModel, path="/home/brett/")


In [16]:
#bestModel = h2o.load_model(path="/home/brett/ticker/h2ogbm.h2o")

In [17]:
importances = bestModel.varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

In [29]:
cm = bestModel.confusion_matrix(valid=True)
cm = cm.table.as_data_frame()
confusionMatrix = FF.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

In [22]:
pBestModel = pd.concat([bestModel.predict(test_data=testReduced).as_data_frame(use_pandas=True).reset_index(drop=True), testDF.reset_index(drop=False)], axis =1)

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [23]:
prices = df.loc[df['Date'] > split_test]
prices = prices.loc[:,['Date','Close','Close Tomorrow','Close Tomorrow Trend']]
profit = pBestModel.loc[:,['predict']]
profit = pd.concat([prices.reset_index(drop=True), profit.reset_index(drop=True)], axis =1)
profit['predict'] = profit['predict'].astype(np.float64)

profit['profit'] = invest

profit['Best Possible'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['Close Tomorrow Trend'] > 0 else 0, axis=1)

profit['Randomly Invest'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if random.randint(1,1000) > random.randint(1,1000) else 0, axis=1)
profit['profit'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['predict'] > 0 else 0, axis=1)


print "Times " + stockSymbol + " actually went up    : " + str(profit[profit['Close Tomorrow Trend']>0].count()['Close Tomorrow Trend'])
print "Times " + stockSymbol + " predicted to go up  : " + str(profit[profit['predict']>0].count()['predict'])
print "Profit                        : $" + str(profit.profit.sum())
print "Random investment             : $" + str(profit['Randomly Invest'].sum())
print "Best possible outcome         : $" + str(profit['Best Possible'].sum())

profit



Times GOOG actually went up    : 46
Times GOOG predicted to go up  : 73
Profit                        : $-524.4155140382409
Random investment             : $-101.78125221451228
Best possible outcome         : $5673.069507204


,Date,Close,Close Tomorrow,Close Tomorrow Trend,predict,profit,Best Possible,Randomly Invest
0,2018-01-02,1065.00,1082.48,1.0,1.0,164.131455,164.131455,164.131455
1,2018-01-03,1082.48,1086.40,1.0,-1.0,0.000000,36.213140,0.000000
2,2018-01-04,1086.40,1102.23,1.0,1.0,145.710604,145.710604,145.710604
3,2018-01-05,1102.23,1106.94,1.0,1.0,42.731553,42.731553,42.731553
4,2018-01-08,1106.94,1106.26,-1.0,-1.0,0.000000,0.000000,-6.143061
5,2018-01-09,1106.26,1102.61,-1.0,1.0,-32.994052,0.000000,0.000000
6,2018-01-10,1102.61,1105.52,1.0,-1.0,0.000000,26.391925,26.391925
7,2018-01-11,1105.52,1122.26,1.0,-1.0,0.000000,151.421955,151.421955
8,2018-01-12,1122.26,1122.26,-1.0,1.0,0.000000,0.000000,0.000000
9,2018-01-15,1122.26,1121.76,-1.0,-1.0,0.000000,0.000000,-4.455296


In [21]:
bestModel


Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  test8_model_0


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.297846105319
RMSE: 0.545752787733
LogLoss: 0.868479625957
Mean Per-Class Error: 0.432843137255
AUC: 0.572017630582
Gini: 0.144035261163
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.00792400515627: 


,-1,1,Error,Rate
-1,0.0,357.0,1.0,(357.0/357.0)
1,0.0,340.0,0.0,(0.0/340.0)
Total,0.0,697.0,0.5122,(357.0/697.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0079240,0.6557377,399.0
max f2,0.0079240,0.8264463,399.0
max f0point5,0.1999006,0.5563910,325.0
max accuracy,0.7810424,0.5724534,90.0
max precision,0.9899528,1.0,0.0
max recall,0.0079240,1.0,399.0
max specificity,0.9899528,1.0,0.0
max absolute_mcc,0.7810424,0.1546766,90.0
max min_per_class_accuracy,0.5426413,0.5470588,190.0
max mean_per_class_accuracy,0.7572475,0.5671569,105.0


Gains/Lift Table: Avg response rate: 48.78 %



...


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-04-30 01:20:04,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-04-30 01:20:05,0.649 sec,6324 obs/sec,5.0,1,3485.0,0.5457528,0.8684796,0.5720176,1.7571429,0.5121951,0.5245129,0.7928556,0.6042854,1.2083333,0.4022989
,2018-04-30 01:20:09,5.391 sec,6636 obs/sec,50.0,10,34850.0,0.4854228,0.6646029,0.6706335,2.05,0.4476327,0.5080350,0.7128159,0.5725309,1.8125,0.4099617
,2018-04-30 01:20:09,5.440 sec,6631 obs/sec,50.0,10,34850.0,0.5457528,0.8684796,0.5720176,1.7571429,0.5121951,0.5245129,0.7928556,0.6042854,1.2083333,0.4022989


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
PC3,1.0,1.0,0.0254063
PC6,0.8931712,0.8931712,0.0226922
PC40,0.8764411,0.8764411,0.0222672
PC10,0.8729314,0.8729314,0.0221780
PC4,0.8559939,0.8559939,0.0217477
---,---,---,---
PC35,0.7219192,0.7219192,0.0183413
PC5,0.7192584,0.7192584,0.0182737
PC11,0.7109153,0.7109153,0.0180618
PC44,0.7106254,0.7106254,0.0180544



See the whole table with table.as_data_frame()
